In [25]:
import numpy as np
import pandas as pd
from PIL import Image
from skimage import filters
import scipy.ndimage as sci_image
from skimage import measure as meas
import matplotlib.pyplot as plt
from scipy import signal as sci_signal
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import os
import warnings
warnings.filterwarnings("ignore")
import supported_functions as sf

In [27]:
char_list = list(range(48,58)) + list(range(65,91))
for i in char_list:
    file_name = "data/" + chr(i) + ".bmp"
    print(sf.detect_letters(cv2.imread(file_name)))

0
1
2
3
4
5
6
7
8
9
A
B
C
D
E
F
G
H
I
J
K
L
M
N
0
P
Q
R
S
T
U
V
W
X
Y
Z


In [23]:
image = cv2.imread("test/image1.png")
image_scale = image / 255.0
image_grayscale = cv2.cvtColor(image_scale, cv2.COLOR_BGR2GRAY)

error: OpenCV(4.9.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:94: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x59191d0d::Set<1,-1,-1>,struct cv::impl::A0x59191d0d::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Unsupported depth of input image:
>     'VDepth::contains(depth)'
> where
>     'depth' is 6 (CV_64F)


In [24]:
image_grayscale

array([[131, 131, 131, ..., 137, 137, 137],
       [130, 130, 130, ..., 137, 137, 137],
       [130, 130, 130, ..., 137, 137, 137],
       ...,
       [ 24,  24,  24, ...,  28,  28,  29],
       [ 25,  25,  25, ...,  29,  29,  29],
       [ 25,  25,  25, ...,  29,  29,  29]], dtype=uint8)